In [20]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle



In [47]:
## load datset
data = pd.read_csv('Churn_Modelling.csv')
data.head()



,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [48]:
# Preprocess the data
data = data.drop(columns=['RowNumber', 'CustomerId', 'Surname'], axis=1)
data.head()


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [49]:
##Encode categorical variables
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])
data.head()



,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [50]:
## One Hot Encoding for 'Geography' column
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo = OneHotEncoder(sparse_output=False)
goe_encoder = onehot_encoder_geo.fit_transform(data[['Geography']])
geo_df = pd.DataFrame(goe_encoder, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
data = pd.concat([data, geo_df], axis=1)
data = data.drop('Geography', axis=1)
data.head()


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [51]:
## save the endcoders and scalers
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)
with open('onehot_encoder_geo.pkl', 'wb') as file:    
    pickle.dump(onehot_encoder_geo, file)

In [53]:
## split the data into features and target
X = data.drop('Exited', axis=1)
y = data['Exited']
X.head()
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Continue with scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [74]:
##save the scaler
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

## ANN implementation

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
import datetime

In [60]:
## build the ANN model
model = Sequential([
    Dense(units=64, activation='relu', input_shape=(X_train.shape[1],)),  ## FIRST HIDDEN LAYER
    Dense(units=32, activation='relu'),  ## SECOND HIDDEN LAYER  
    Dense(units=1, activation='sigmoid')  ## OUTPUT LAYER
]
)

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 64)                832       
                                                                 
 dense_3 (Dense)             (None, 32)                2080      
                                                                 
 dense_4 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [64]:
##otherwise compile the model
import tensorflow
opt = tensorflow.keras.optimizers.Adam(learning_rate=0.001)
loss= tensorflow.keras.losses.BinaryCrossentropy()
loss

In [65]:
##compile the model
model.compile(optimizer=opt, loss=loss, metrics=['accuracy'])


In [ ]:
## set up tensorboard for training visualization
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [68]:
# Define early stopping callback
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [69]:
## train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    batch_size=32,
    callbacks=[tensorboard_callback, early_stopping_callback]
) 

Epoch 1/100


250/250 [==============================] - 7s 11ms/step - loss: 0.4598 - accuracy: 0.7961 - val_loss: 0.4036 - val_accuracy: 0.8255
Epoch 2/100
250/250 [==============================] - 2s 8ms/step - loss: 0.3925 - accuracy: 0.8357 - val_loss: 0.3600 - val_accuracy: 0.8460
Epoch 3/100
250/250 [==============================] - 2s 9ms/step - loss: 0.3596 - accuracy: 0.8564 - val_loss: 0.3463 - val_accuracy: 0.8555
Epoch 4/100
250/250 [==============================] - 2s 9ms/step - loss: 0.3466 - accuracy: 0.8590 - val_loss: 0.3416 - val_accuracy: 0.8580
Epoch 5/100
250/250 [==============================] - 2s 9ms/step - loss: 0.3409 - accuracy: 0.8608 - val_loss: 0.3436 - val_accuracy: 0.8585
Epoch 6/100
250/250 [==============================] - 2s 9ms/step - loss: 0.3380 - accuracy: 0.8593 - val_loss: 0.3486 - val_accuracy: 0.8575
Epoch 7/100
250/250 [==============================] - 2s 7ms/step - loss: 0.3354 - accuracy: 0.8619 - val_loss: 0.3397 - val_accuracy: 0.8

In [70]:
##save the trained model
model.save('model.h5')


c:\ANN PROJE T\nenv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [72]:
## load tensorboard extension
%load_ext tensorboard




The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [73]:
%tensorboard --logdir logs/fit


Reusing TensorBoard on port 6006 (pid 12692), started 0:02:26 ago. (Use '!kill 12692' to kill it.)

In [ ]:
## load pickle files
with open('model.h5', 'rb') as file:
    loaded_model = tf.keras.models.load_model